In [ ]:
from robyn.modeling.clustering.clustering_config import ClusteringConfig, ClusterBy
from robyn.modeling.clustering.cluster_builder import ClusterBuilder
from robyn.modeling.entities.pareto_result import ParetoResult
from robyn.data.entities.holidays_data import HolidaysData
from robyn.data.entities.enums import DependentVarType
import logging
# Set the log level to DEBUG
import os
import logging
log_level = os.environ.get('LOG_LEVEL', 'INFO')
logging.basicConfig(level=getattr(logging, log_level.upper()))


In [2]:

import pickle

# load the variable from the file
with open("pareto_result.pkl", "rb") as f:
    pareto_result: ParetoResult = pickle.load(f)
with open("mmmdata.pkl", "rb") as f:
    mmm_data = pickle.load(f)
with open("holidays_data.pkl", "rb") as f:
    holidays_data:HolidaysData = pickle.load(f)


In [ ]:
cluster_builder = ClusterBuilder(pareto_result=pareto_result)

In [ ]:
configs = ClusteringConfig(
    dep_var_type= DependentVarType(mmm_data.mmmdata_spec.dep_var_type),
    cluster_by = ClusterBy.HYPERPARAMETERS,
    max_clusters = 10,
    min_clusters = 3,
    weights=[1.0, 1.0, 1.0]
)
cluster_results  = cluster_builder.cluster_models(configs)

In [ ]:
cluster_results.wss

In [ ]:
cluster_results.plots.top_solutions_errors_plot

In [ ]:
cluster_results.plots.top_solutions_rois_plot

In [ ]:
cluster_results.cluster_ci.clusters_confidence_interval_plot

In [ ]:
print("clusters: ", cluster_results.n_clusters,
"\nerror weights/balance: ", cluster_results.errors_weights,
"\nboot_n: ", cluster_results.cluster_ci.boot_n,
"\nsim_n: ", cluster_results.cluster_ci.sim_n)


In [ ]:
print(cluster_results.cluster_ci.cluster_confidence_interval_df)

In [ ]:
cluster_results.top_solutions['sol_id']

In [12]:
from robyn.modeling.pareto.pareto_utils import ParetoUtils

utils = ParetoUtils()
new_pareto_results = utils.process_pareto_clustered_results(pareto_result, clustered_result=cluster_results, ran_cluster=True, ran_calibration= False)

In [ ]:
from robyn.data.entities.enums import AdstockType   
from robyn.data.entities.hyperparameters import Hyperparameters
from robyn.reporting.onepager_reporting import OnePager

visualizer = OnePager(pareto_result=new_pareto_results, clustered_result=cluster_results, hyperparameter=Hyperparameters(adstock=AdstockType.GEOMETRIC), mmm_data=mmm_data, holidays_data=holidays_data)
visualizer.generate_one_pager(top_pareto=True)